In [1]:
import os
import numpy as np
from glob import glob
from tqdm import tqdm
from openai import OpenAI
import traceback
import re
from pymilvus import MilvusClient, model as milvus_model
import time

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("请设置环境变量 DEEPSEEK_API_KEY")

# 初始化 DeepSeek 客户端 (仅用于问答)
deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",
    timeout=30
)

# 初始化 Milvus 嵌入模型
embedding_model = milvus_model.DefaultEmbeddingFunction()

# 测试嵌入模型
test_embedding = embedding_model.encode_queries(["测试文本"])[0]
embedding_dim = len(test_embedding)
print(f"嵌入向量维度: {embedding_dim}")
print(f"示例嵌入: {test_embedding[:5]}...")

# 初始化 Milvus 客户端 (嵌入式模式)
milvus_client = MilvusClient(uri="./milvus_demo.db")  # 本地数据库文件

# 集合名称
collection_name = "civil_code_rag"

/home/parallels/miniconda3/envs/deepseek/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


嵌入向量维度: 768
示例嵌入: [-0.00993222  0.04682298 -0.03675228 -0.08068266 -0.02617284]...


In [2]:
# 1. 文档分割策略
def load_and_split_documents(file_pattern):
    """加载文档并按民法典条文结构分割"""
    print(f"加载法律文档: {file_pattern}...")
    file_paths = glob(file_pattern, recursive=False)
    
    if not file_paths:
        raise FileNotFoundError(f"未找到匹配的法律文件: {file_pattern}")
    
    text_chunks = []
    for file_path in file_paths:
        print(f"处理: {os.path.basename(file_path)}")
        
        with open(file_path, "r", encoding="utf-8") as file:
            file_text = file.read()
            
        # 使用条文编号作为分割点
        articles = re.split(r'(\*\*第[\d零一二三四五六七八九十百千]+条\*\*)', file_text)
        
        # 重建条文结构
        chunks = []
        current_chunk = ""
        for i, part in enumerate(articles):
            if re.match(r'\*\*第[\d一二三四五六七八九十百千]+条\*\*', part):  # 简化正则表达式
                if current_chunk:
                    chunks.append(current_chunk.strip())
                current_chunk = part
            else:
                current_chunk += part
                
        if current_chunk:
            chunks.append(current_chunk.strip())
        
        # 进一步分割过大的块（超过2000字）
        final_chunks = []
        for chunk in chunks:
            if len(chunk) > 2000:
                # 使用自然段落分割
                paragraphs = re.split(r'(\n{2,})', chunk)
                for para in paragraphs:
                    if para.strip() and not re.match(r'\n+', para):
                        final_chunks.append(para.strip())
            else:
                final_chunks.append(chunk)
                
        text_chunks.extend(final_chunks)
    
    print(f"总计分割出 {len(text_chunks)} 个法律条文")
    return text_chunks


In [3]:
# 2. 初始化向量数据库 - 修复实体数量获取问题
def initialize_vector_database(chunks):
    """创建并填充向量数据库"""
    # 检查并删除现有集合
    if milvus_client.has_collection(collection_name):
        print(f"删除现有集合: {collection_name}")
        milvus_client.drop_collection(collection_name)
        time.sleep(1)  # 稍作等待确保操作完成
    
    # 创建新集合
    print(f"创建新集合: {collection_name}")
    result = milvus_client.create_collection(
        collection_name=collection_name,
        dimension=embedding_dim,
        auto_id=True
    )
    print(f"集合创建结果: {result}")
    
    # 确保集合已存在
    while not milvus_client.has_collection(collection_name):
        print("等待集合创建完成...")
        time.sleep(1)
    
    # 生成嵌入向量
    print(f"生成文本嵌入...")
    embeddings = []
    texts = []
    article_numbers = []
    
    # 分批处理以避免内存问题
    batch_size = 100
    for i in tqdm(range(0, len(chunks), batch_size), desc="生成嵌入"):
        batch = chunks[i:i+batch_size]
        batch_embeddings = embedding_model.encode_documents(batch)
        embeddings.extend(batch_embeddings)
        texts.extend(batch)
        
        # 提取条文编号
        for text in batch:
            article_match = re.search(r"\*\*第([零一二三四五六七八九十百千]+)条\*\*", text)
            article_numbers.append(article_match.group(1) if article_match else "")
    
    print(f"嵌入生成完成")
    
    # 准备插入数据
    data = []
    for text, emb, art_num in zip(texts, embeddings, article_numbers):
        data.append({
            "text": text,
            "article_number": art_num,
            "vector": emb
        })
    
    # 分批插入数据库
    print(f"插入数据到向量数据库...")
    total_inserted = 0
    insert_batch_size = 100
    for i in tqdm(range(0, len(data), insert_batch_size), desc="插入数据"):
        batch = data[i:i+insert_batch_size]
        result = milvus_client.insert(collection_name, batch)
        total_inserted += result['insert_count']
        print(f"已插入 {total_inserted}/{len(data)} 条数据")
    
    # 检查是否所有数据都已插入
    if total_inserted == len(data):
        print(f"✅ 成功插入所有 {total_inserted} 条文数据")
    else:
        print(f"⚠️ 插入异常: 预期 {len(data)} 条, 实际插入 {total_inserted} 条")
    
    # 获取文档数量
    try:
        # 尝试获取实体数量
        num_entities = milvus_client.num_entities(collection_name)
        print(f"集合文档数: {num_entities}")
        return num_entities
    except Exception as e:
        # 如果无法获取实体数量，返回已知数据量
        print(f"无法获取集合实体数量: {e}")
        print(f"使用插入数据量: {total_inserted}")
        return total_inserted


In [4]:
# 3. 向量搜索函数
def vector_search(query, top_k=5):
    """执行向量搜索"""
    try:
        # 生成查询向量
        query_embedding = embedding_model.encode_queries([query])[0]
        
        # 执行搜索
        results = milvus_client.search(
            collection_name=collection_name,
            data=[query_embedding],
            output_fields=["text", "article_number"],
            limit=top_k
        )
        
        # 格式化结果
        formatted_results = []
        for hit in results[0]:
            text = hit["entity"]["text"]
            article_number = hit["entity"]["article_number"]
            similarity = hit["distance"]
            
            # 提取标题
            if article_number:
                title_match = re.search(rf"\*\*第{article_number}条\*\* ([^\n]+)", text)
                title = title_match.group(1) if title_match else text[:50] + "..."
                title = f"第{article_number}条: {title}"
            else:
                title = text[:50] + "..."
            
            formatted_results.append({
                "text": text,
                "article_number": article_number,
                "similarity": similarity,
                "title": title
            })
        
        return formatted_results
    except Exception as e:
        # 搜索失败时返回空结果
        print(f"向量搜索失败: {e}")
        return []

In [5]:
# 4. 法律问答系统
def legal_qa_system():
    print("=" * 60)
    print(" 中华人民共和国民法典智能法律顾问系统 ")
    print("=" * 60)
    
    # 1. 加载法律文档
    try:
        documents = load_and_split_documents("mfd.md")
        if not documents:
            raise ValueError("未能加载任何法律条文")
        print(f"成功加载 {len(documents)} 个法律条文")
    except Exception as e:
        print(f"法律条文加载失败: {e}")
        traceback.print_exc()
        return
    
    # 2. 初始化向量数据库
    print("\n" + "━" * 60)
    print(" 构建民法典知识库...")
    num_documents = initialize_vector_database(documents)
    print("\n" + "━" * 60)
    print(f"✅ 民法典知识库准备完成，包含 {num_documents} 个条文")
    print("━" * 60)
    
    # 3. 用户咨询交互
    while True:
        try:
            print("\n" + "-" * 60)
            question = input("请输入您的法律问题（输入'退出'结束咨询）: ")
            
            if question.lower() in ['exit', 'quit', '退出']:
                print("感谢使用《民法典》智能法律顾问！")
                break
                
            if not question.strip():
                print("问题内容不能为空，请重新输入。")
                continue
                
            print(f"\n🔍 正在分析: 「{question}」")
            
            # 检索相关条文
            print("正在检索相关法律条文...")
            search_results = vector_search(question, top_k=5)
            
            if not search_results:
                print("\n⚠️ 未检索到相关法律条文，将尝试直接回答问题")
                search_results = []  # 确保列表为空
                context_str = ""
            else:
                # 显示检索结果
                print("\n相关条文检索结果:")
                for i, result in enumerate(search_results):
                    print(f"  {i+1}. {result['title']} | 相关度: {result['similarity']:.3f}")
                
                # 准备法律问答提示
                context = []
                for i, result in enumerate(search_results):
                    # 提取条文内容的前200个字符
                    content = result['text'].replace("**", "").strip()
                    content = re.sub(r'\n{3,}', '\n\n', content)  # 减少空行
                    content = content[:400] + "..." if len(content) > 400 else content
                    context.append(f"相关条文 {i+1} (相关度: {result['similarity']:.3f}):\n{content}")
                
                context_str = "\n\n".join(context)
            
            # 构建提示词
            if context_str:
                prompt = f"""
                ### 用户法律咨询
                问题: {question}
                
                ### 相关法律条文参考
                {context_str}
                
                ### 回答要求
                请基于上述相关法律条文，严格遵循以下要求回答问题：
                1. 准确引用具体条文（格式：根据第XXX条）
                2. 解释法律概念和适用范围
                3. 结合用户问题分析权利义务
                4. 使用专业法律术语但避免过度学术化
                """
            else:
                prompt = f"""
                ### 用户法律咨询
                问题: {question}
                
                ### 回答要求
                作为《中华人民共和国民法典》专业法律顾问，请针对上述问题提供专业法律意见。
                即使没有直接相关条文，也请基于民法典的基本原则提供合理建议。
                """
            
            # 调用DeepSeek生成专业法律意见
            print("\n📝 生成专业法律分析中...")
            response = deepseek_client.chat.completions.create(
                model="deepseek-chat",
                messages=[
                    {"role": "system", "content": "你是一名专业的法律顾问，专门回答《民法典》相关问题。请确保回答准确、专业且有帮助。"},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.2,
                max_tokens=1500
            )
            
            # 输出最终法律意见
            legal_opinion = response.choices[0].message.content
            print("\n" + "="*60)
            print("📜 专业法律意见:")
            print(legal_opinion)
            print("="*60)
            
        except KeyboardInterrupt:
            print("\n咨询已终止。")
            break
        except Exception as e:
            print(f"\n处理咨询时出错: {str(e)[:200]}")
            print("请尝试重新表述您的问题或咨询专业律师。")


In [ ]:
# 启动法律咨询系统
if __name__ == "__main__":
    legal_qa_system()

 中华人民共和国民法典智能法律顾问系统 
加载法律文档: mfd.md...
处理: mfd.md
总计分割出 355 个法律条文
成功加载 355 个法律条文

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
 构建民法典知识库...
创建新集合: civil_code_rag
集合创建结果: None
生成文本嵌入...


生成嵌入: 100%|███████████████████████████████████| 4/4 [02:12<00:00, 33.07s/it]


嵌入生成完成
插入数据到向量数据库...


插入数据:  25%|████████▊                          | 1/4 [00:00<00:00,  6.00it/s]

已插入 100/355 条数据
已插入 200/355 条数据
已插入 300/355 条数据


...

已插入 355/355 条数据
✅ 成功插入所有 355 条文数据
无法获取集合实体数量: 'MilvusClient' object has no attribute 'num_entities'
使用插入数据量: 355

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅ 民法典知识库准备完成，包含 355 个条文
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

------------------------------------------------------------


请输入您的法律问题（输入'退出'结束咨询）:  房屋拆迁后，宅基地的归属？



🔍 正在分析: 「房屋拆迁后，宅基地的归属？」
正在检索相关法律条文...

相关条文检索结果:
  1. 第二百二十九条: 因买卖、设定担保物权等基于法律行为而取得物权的，自法律行为生效时发生效力，但是法律另有规定的除外。 | 相关度: 0.688
  2. 第二百二十五条: 动产物权设立和转让前，权利人已经占有该动产的，物权自民事法律行为生效时发生效力。 | 相关度: 0.688
  3. 第二百二十四条: 船舶、航空器和机动车等的物权的设立、变更、转让和消灭，未经登记，不得对抗善意第三人。 | 相关度: 0.688
  4. 第二百二十三条: 动产物权的设立和转让，自交付时发生效力，但是法律另有规定的除外。 | 相关度: 0.688
  5. 第二百一十四条: 不动产物权的设立、变更、转让和消灭，依照法律规定应当登记的，自记载于不动产登记簿时发生效力。 | 相关度: 0.688

📝 生成专业法律分析中...

📜 专业法律意见:
关于房屋拆迁后宅基地的归属问题，根据《民法典》及相关法律规定，分析如下：

1. 权属确认依据
根据第214条："不动产物权的设立、变更、转让和消灭，依照法律规定应当登记的，自记载于不动产登记簿时发生效力。"宅基地作为集体建设用地，其权属应以不动产登记簿记载为准。

2. 拆迁补偿原则
宅基地使用权具有特殊性：
- 所有权归农村集体经济组织（第261条）
- 使用权归符合宅基地分配条件的农户
- 房屋所有权与宅基地使用权相分离

3. 拆迁后处理规则
（1）宅基地补偿款归属：
- 土地补偿款归集体（需经成员会议决定用途）
- 地上附着物补偿归房屋所有权人

（2）重新分配：
- 符合宅基地申请条件的，可申请新宅基地
- 已进行安置补偿的，原宅基地使用权消灭（第358条）

4. 实务要点
- 需核查当地集体土地征收补偿办法
- 注意区分"征地补偿"与"房屋拆迁补偿"
- 宅基地使用权不得单独转让（第363条）

建议补充说明：
1. 被拆迁人是否仍具有集体经济组织成员资格
2. 当地是否实行"宅基地退出"政策
3. 拆迁补偿协议的具体约定

如需进一步分析，请提供：
- 宅基地登记情况
- 拆迁补偿方案
- 家庭成员户籍信息

----------------------------------------------------

请输入您的法律问题（输入'退出'结束咨询）:  在路上拾到钱，该怎么处理？



🔍 正在分析: 「在路上拾到钱，该怎么处理？」
正在检索相关法律条文...

相关条文检索结果:
  1. 第二百二十九条: 因买卖、设定担保物权等基于法律行为而取得物权的，自法律行为生效时发生效力，但是法律另有规定的除外。 | 相关度: 0.688
  2. 第二百二十五条: 动产物权设立和转让前，权利人已经占有该动产的，物权自民事法律行为生效时发生效力。 | 相关度: 0.688
  3. 第二百二十四条: 船舶、航空器和机动车等的物权的设立、变更、转让和消灭，未经登记，不得对抗善意第三人。 | 相关度: 0.688
  4. 第二百二十三条: 动产物权的设立和转让，自交付时发生效力，但是法律另有规定的除外。 | 相关度: 0.688
  5. 第二百一十四条: 不动产物权的设立、变更、转让和消灭，依照法律规定应当登记的，自记载于不动产登记簿时发生效力。 | 相关度: 0.688

📝 生成专业法律分析中...

📜 专业法律意见:
关于拾得遗失物的处理问题，根据《民法典》相关规定，为您作出专业解答：

1. 法律依据：
虽然您提供的条文主要涉及物权变动规则，但拾得遗失物的正确处理应适用《民法典》第314-316条关于遗失物拾得的规定。

2. 法律义务分析：
- 拾得人应当及时通知权利人领取（第314条）
- 应当妥善保管遗失物，因故意或重大过失致损需赔偿（第316条）
- 送交公安机关前，拾得人构成无因管理（第314条）

3. 具体处理建议：
（1）拾得现金后：
① 应尽量在原地等待失主
② 可交至附近公安机关（须索取收据）
③ 不得侵占，否则可能构成不当得利

4. 权利保障：
拾得人有权要求失主支付必要保管费用（第317条），但悬赏广告情形除外。

特别提示：
若拾得物价值较大拒不归还，可能涉嫌侵占罪（《刑法》第271条）。建议通过正规渠道处理，既履行法定义务，也保障自身权益。

（注：您提供的条文主要规范物权变动规则，与遗失物处理关联性较弱，故未直接引用）

------------------------------------------------------------
